**Downloading quickdraw libraries**

In [ ]:
!pip install quickdraw

# **Import libraries**

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization

import numpy as np
from pathlib import Path
from quickdraw import QuickDrawDataGroup
from matplotlib import pyplot as plt
from PIL import Image

**Change the device to GPU**

In [ ]:
# import torch
# from torch._C import device
# # Get cpu or gpu device (if available) for training.
# if torch.cuda.is_available():
#   device = torch.device("cuda")
# print(f"Using {device} device")

In [ ]:
# import numpy as np
# import glob

# directory_path = '/content/Dataset'
# file_paths = glob.glob(directory_path + '/*.npy')

# # Check the shape of each file
# for file_path in file_paths:
#     with open(file_path, 'rb') as f:
#         data = np.load(f, allow_pickle=True)
#         print(f"{file_path}: {data.shape}")


Creat

**Defining the categories**

In [ ]:
categories = ("bat", "bee", "cat", "duck", "elephant", "lion", "octopus", "rabbit", "snail", "whale")
image_size = (64, 64)
max_drawings = 3000

**Getting the doodles**

In [ ]:
image_size = (64, 64)
categories = ["airplane", "apple", "bicycle", "car", "cat", "cloud", "dog", "hamburger", "fish", "flower", "banana", "bird", "eye", "fork", "hat"]
max_drawings = 5000

def generate_class_images(name, max_drawings, recognized):
    directory = Path("data/" + name)

    if not directory.exists():
        directory.mkdir(parents=True)

    images = QuickDrawDataGroup(name, max_drawings=max_drawings, recognized=recognized)
    for img in images.drawings:
        filename = directory.as_posix() + "/" + str(img.key_id) + ".png"
        img.get_image(stroke_width=3).resize(image_size).save(filename)

for label in categories:
    generate_class_images(label, max_drawings=max_drawings, recognized=True)

# **Defining dataset**

In [ ]:
batch_size = 32

train_dataset = image_dataset_from_directory(
    "data",
    validation_split=0.2,
    subset="training",
    seed=123,
    color_mode="grayscale",
    image_size=image_size,
    batch_size=batch_size
)

validation_dataset = image_dataset_from_directory(
    "data",
    validation_split=0.2,
    subset="validation",
    seed=123,
    color_mode="grayscale",
    image_size=image_size,
    batch_size=batch_size
)

In [ ]:
# import torch
# from torch import nn
# from torch.utils.data import DataLoader
# import torchvision
# from torchvision import datasets

# batch_size = 64

# # Create data loaders.
# train_dataloader = DataLoader(X_train, batch_size=batch_size)
# test_dataloader = DataLoader(X_test, batch_size=batch_size)

# # for X, y in test_dataloader:
# #     print(f"Shape of X [N, C, H, W]: {X.shape}")
# #     print(f"Shape of y: {y.shape} {y.dtype}")
# #     break

# **Creating the model**

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.optimizers import SGD
# def softmax(x):
#   return torch.exp(x) / torch.sum(torch.exp(x), dim=0)

# loss_fn = tf.keras.metrics.categorical_crossentropy
# lr = 5e-1
# optm = SGD(learning_rate=lr)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

num_classes = 10  # Set this to the number of categories you have

model = tf.keras.Sequential([
    Rescaling(1. / 255, input_shape=(64, 64, 1)),
    BatchNormalization(),

    Conv2D(6, kernel_size=(3, 3), padding="same", activation="relu"),
    Conv2D(8, kernel_size=(3, 3), padding="same", activation="relu"),
    Conv2D(10, kernel_size=(3, 3), padding="same", activation="relu"),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),

    Dense(700, activation="relu"),
    BatchNormalization(),
    Dropout(0.2),

    Dense(500, activation="relu"),
    BatchNormalization(),
    Dropout(0.2),

    Dense(400, activation="relu"),
    Dropout(0.2),

    Dense(len(categories), activation="softmax")
])

# **Compiling the model**

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# **Training the model**

In [ ]:
epochs = 32

model.fit(
    train_dataset,
    validation_data = validation_dataset,
    epochs = epochs
)

model.save("./models/test_model")

In [ ]:
# def train(dataloader, model, loss_fn, optimizer):
#     size = len(dataloader.dataset)
#     # count the correct predictions
#     correct = 0
#     # when training, we put the model in train mode
#     model.train()
#     # we iterate over the dataloader
#     for batch, (X, y) in enumerate(dataloader):
#         # in each iteration, we work with a batch of 64 images
#         # we move to GPU first if GPU is available
#         X, y = X.to(device), y.to(device)

#         # step1: forward pass
#         ### Your code here (1 line) ###
#         pred = model(X)

#         ######################

#         # step2: compute prediction error/loss
#         ### Your code here (1 line) ###
#         loss = loss_fn(pred, y)
#         ######################

#         # counting the correct predictions
#         correct += (pred.argmax(1) == y).type(torch.float).sum().item()

#         # step 3: Backpropagation & parameter updating
#         loss.backward()
#         optimizer.step()


#         ######################

#         # step 4: zero the accumulated gradients in the tensors (1 lines)
#         optimizer.zero_grad()


#         ######################


#         # every 100 batches, we print out the loss information to get an idea of
#         # how good we are
#         if batch % 100 == 0:
#             loss, current = loss.item(), (batch + 1) * len(X)
#             print(f"Running loss for a batch of 100 images: {loss:>7f}  [{current:>5d}/{size:>5d}]")
#     correct /= size
#     print(f"Training accuracy for this epoch: {(100*correct):>0.1f}%")

In [ ]:
# epochs = 30
# batch_size = 64
# model.fit(train_dataloader,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=test_dataloader)

# **Testing accuracy**

In [ ]:
test_loss, test_acc = model.evaluate(validation_dataset, verbose=2)

**Input a random image**

In [ ]:
for images, labels in train_dataset.take(1):
  data = images[0].numpy().astype("uint8")
  plt.imshow(data, cmap='gray', vmin=0, vmax=255)
  plt.title(train_dataset.class_names[labels[0]])
  plt.axis("off")

**Make a prediction**

In [ ]:
prediction_data = train_dataset.take(1)
for images, labels in prediction_data:
  data = images[0].numpy().astype("uint8")
  plt.imshow(data, cmap='gray', vmin=0, vmax=255)
  plt.title(train_dataset.class_names[labels[0]])
  plt.axis("off")


predictions = model_3.predict(prediction_data)
categories[np.argmax(predictions[0])]

In [ ]:
# def test(dataloader, model, loss_fn):
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     # when evaluating, we put the model in evaluation mode
#     model.eval()
#     test_loss, correct = 0, 0
#     # when evaluating, we disable gradient accumulations
#     with torch.no_grad():
#         for X, y in dataloader:
#             X, y = X.to(device), y.to(device)
#             pred = model(X)
#             test_loss += loss_fn(pred, y).item()
#             # this compute the accuracy
#             correct += (pred.argmax(1) == y).type(torch.float).sum().item()
#     test_loss /= num_batches
#     correct /= size
#     print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
# test_loss, test_acc = model.evaluate(X_test, y_test)
# print("Test Accuracy:", test_acc)


In [ ]:
# torch.save(model.state_dict(), "model.pth")
# # save the state_dict to the path "model.pth"
# # TODO
# print("Saved PyTorch Model State to model.pth")

In [ ]:
# test(test_dataloader, model, loss_fn)